In [1]:
# read the text file "NE.population" and store in the list called population
population = list()

# open the text file for reading
filepath = "C:\\Users\\jayde\\Downloads\\"
filename = "NE.population .Copy.txt"
file = open( filepath + filename ,"r")

# while the current line is not empty, read in a new county population
line = file.readline()


while line != "":
    # split the line into two "words": 
    #    word[0]: the county's number
    #    word[1]: the county's population
    words = line.split() 
    county_number = words[0]
    county_population = int(words[1]) # cast the string as type int
    
    # append to population list
    population.append(county_population)
    
    # read next line
    line = file.readline() 

file.close()
print("population = ",population)

population =  [6129, 9998, 11308, 818, 7547, 15740, 36288, 736, 11055, 614, 36691, 3966, 158840, 36970, 20234, 2538, 3812, 517110, 25241, 5713, 5505, 5217, 9182, 2756, 4500, 6489, 6000, 6274, 824, 763, 8852, 632, 5406, 478, 3821, 2044, 3423, 9188, 9124, 8395, 10435, 32237, 5890, 10939, 46102, 6858, 24326, 20780, 647, 6685, 2049, 1311, 1941, 2773, 2099, 5228, 2908, 690, 4959, 6542, 7845, 3145, 10515, 21006, 460, 5469, 16750, 9595, 8701, 7248, 5042, 58607, 4260, 34876, 3225, 3735, 967, 22311, 13665, 539, 2970, 1526, 285407, 31364, 9139, 2057, 8368, 6940, 7266, 8363, 14200, 3152, 2008]


In [3]:
# Read the Nebraska county graph. It is stored in the file "Ne.graph", with edges listed like this:
# 0 23 
# 0 30 
# 0 47 
# 0 76 
# 1 5 
# 1 8 
# 1 23 
# 1 33 
# ...

# County-level graphs for all states can be found here:
# https://github.com/AustinLBuchanan/county-level-districting/tree/master/data

# NetworkX has a built-in function for reading this kind of file
import networkx as nx
filepath = 'C:\\Users\\jayde\\Downloads\\'
filename = 'Ne.graph.txt'
G = nx.read_edgelist( filepath + filename, nodetype=int )

print("The Nebraska (county) graph has",G.number_of_nodes(),"nodes and",G.number_of_edges(),"edges.")
print("The Nebraska graph has nodes",G.nodes)
print("The Nebraska graph has edges",G.edges)

The Nebraska (county) graph has 93 nodes and 231 edges.
The Nebraska graph has nodes [0, 41, 62, 67, 73, 84, 1, 34, 52, 57, 70, 85, 2, 13, 22, 51, 65, 3, 15, 20, 40, 49, 50, 72, 4, 55, 77, 90, 5, 18, 21, 69, 82, 6, 23, 29, 43, 46, 76, 79, 80, 86, 7, 9, 19, 48, 64, 8, 56, 58, 10, 14, 17, 39, 45, 47, 11, 92, 12, 27, 75, 91, 16, 24, 25, 59, 74, 83, 28, 33, 61, 53, 35, 42, 37, 44, 26, 30, 63, 87, 60, 71, 54, 81, 68, 88, 31, 32, 38, 78, 36, 66, 89]
The Nebraska graph has edges [(0, 41), (0, 62), (0, 67), (0, 73), (0, 84), (41, 20), (41, 32), (41, 39), (41, 60), (41, 62), (41, 73), (41, 75), (62, 10), (62, 39), (62, 84), (67, 26), (67, 30), (67, 84), (67, 87), (67, 88), (73, 20), (73, 49), (73, 88), (84, 10), (84, 45), (84, 87), (1, 34), (1, 52), (1, 57), (1, 70), (1, 85), (34, 57), (52, 80), (52, 85), (52, 86), (57, 13), (57, 70), (70, 2), (70, 13), (70, 65), (70, 85), (85, 9), (85, 64), (85, 65), (85, 86), (2, 13), (2, 22), (2, 51), (2, 65), (13, 51), (22, 51), (22, 65), (65, 9), (65, 19),

In [4]:
# We are to solve the following task:
# input: a population vector, a graph G=(V,E), desired number of districts k, population lower and upper bounds L and U
# output: a partition of the nodes into k districts (not necessarily connected!) 
#            each with population in [L,U] to minimize the number of "cut edges" 
#
# An edge {i,j} from E is cut if its endpoints i and j are assigned to different districts.
#
# For example, consider the 4x4 grid graph: 
#
#         & - & - & - &
#         |   |   |   |
#         & - & - & - &
#         |   |   |   |
#         & - & - & - &
#         |   |   |   |
#         & - & - & - &
#
# Here are two ways to split it into 4 equal-size districts:
#
#         &   &   &   &                & - &   & - &
#         |   |   |   |                |   |   |   |
#         &   &   &   &                & - &   & - &
#         |   |   |   |                             
#         &   &   &   &                & - &   & - &
#         |   |   |   |                |   |   |   |
#         &   &   &   &                & - &   & - &
#
#          12 cut edges                 8 cut edges
#
# The plan with 8 cut edges looks more compact.
#

import gurobipy as gp
from gurobipy import GRB

In [5]:
# Let's impose a 1% population deviation (+/- 0.5%)
deviation = 0.01

import math
k = 3          # number of districts
L = math.ceil((1-deviation/2)*sum(population)/k)
U = math.floor((1+deviation/2)*sum(population)/k)
print("Using L =",L,"and U =",U,"and k =",k)

Using L = 605737 and U = 611824 and k = 3


In [6]:
# create model 
m = gp.Model()

# create variables
x = m.addVars(G.nodes, k, vtype=GRB.BINARY) # x[i,j] equals one when county i is assigned to district j
y = m.addVars(G.edges, vtype=GRB.BINARY)  # y[u,v] equals one when edge {u,v} is cut

Academic license - for non-commercial use only - expires 2021-06-22
Using license file C:\Users\jayde\gurobi.lic


In [7]:

# objective is to minimize cut edges
m.setObjective( gp.quicksum( y[u,v] for u,v in G.edges ), GRB.MINIMIZE )

In [8]:

# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum(x[i,j] for j in range(k)) == 1 for i in G.nodes)

# add constraints saying that each district has population at least L and at most U
m.addConstrs( gp.quicksum( population[i] * x[i,j] for i in G.nodes) >= L for j in range(k) )
m.addConstrs( gp.quicksum( population[i] * x[i,j] for i in G.nodes) <= U for j in range(k) )

# add constraints saying that edge {i,j} is cut if i is assigned to district v but j is not.
m.addConstrs( x[i,v] - x[j,v] <= y[i,j] for i,j in G.edges for v in range(k))

m.update()

In [9]:
# solve IP model
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 792 rows, 510 columns and 2916 nonzeros
Model fingerprint: 0xa469c9e1
Variable types: 0 continuous, 510 integer (510 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+05]
Found heuristic solution: objective 139.0000000
Presolve time: 0.00s
Presolved: 792 rows, 510 columns, 2916 nonzeros
Variable types: 0 continuous, 510 integer (510 binary)

Root relaxation: objective 0.000000e+00, 352 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0  279  139.00000    0.00000   100%     -    0s
H    0     0                      84.0000000    0.00000   100%     -    0s
H    0     0   

In [10]:
# print the number of cut edges
print("The number of cut edges is",m.objval)

# retrieve the districts and their populations
districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in range(k)]
district_populations = [ sum(population[i] for i in district) for district in districts ]

# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",districts[j])

The number of cut edges is 19.0
District 0 has population 606676 and contains counties [0, 62, 84, 10, 14, 17, 45]
District 1 has population 609930 and contains counties [4, 55, 77, 90, 5, 18, 21, 69, 82, 39, 47, 12, 53, 42, 66, 89]
District 2 has population 609735 and contains counties [41, 67, 73, 1, 34, 52, 57, 70, 85, 2, 13, 22, 51, 65, 3, 15, 20, 40, 49, 50, 72, 6, 23, 29, 43, 46, 76, 79, 80, 86, 7, 9, 19, 48, 64, 8, 56, 58, 11, 92, 27, 75, 91, 16, 24, 25, 59, 74, 83, 28, 33, 61, 35, 37, 44, 26, 30, 63, 87, 60, 71, 54, 81, 68, 88, 31, 32, 38, 78, 36]


In [11]:
# check if the districts are connected
for j in range(k):
    print("Is district",j,"connected?", nx.is_connected( G.subgraph( districts[j] ) ) )

Is district 0 connected? True
Is district 1 connected? True
Is district 2 connected? True
